In [35]:
import tensorflow as tf

In [36]:
from tensorflow import keras

In [37]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [38]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Activation
from tensorflow.python.keras.optimizers import adam
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [39]:
Mucuri = pd.read_excel('Mucuri_novo.xlsx')

In [40]:
Mucuri[:10]

,Data,dia,mês,ano,hora,v_anemo2,dir_1,temp,umid,pressão
0,2015-11-30,30,11,2015,14,13.012139,75.105481,27.516129,72.930636,1020.422601
1,2015-11-30,30,11,2015,15,12.726087,68.334332,27.238095,75.212121,1020.394348
2,2015-11-30,30,11,2015,16,12.081111,64.457865,27.105263,75.741379,1020.508333
3,2015-11-30,30,11,2015,17,11.647222,53.842100,26.305556,75.302632,1020.611000
4,2015-11-30,30,11,2015,18,11.064444,53.945279,25.464286,76.592593,1020.866500
5,2015-11-30,30,11,2015,19,10.324444,51.320714,24.764706,83.344444,1021.286500
6,2015-11-30,30,11,2015,20,9.862778,47.354507,24.310811,88.044643,1021.627500
7,2015-11-30,30,11,2015,21,9.598889,41.816108,24.304348,91.329670,1021.892333
8,2015-11-30,30,11,2015,22,9.488889,39.561270,24.500000,92.415385,1021.856167
9,2015-11-30,30,11,2015,23,7.502222,37.881537,24.419355,93.629630,1021.834667


In [41]:
Mucuri.describe()

,dia,mês,ano,hora,v_anemo2,dir_1,temp,umid,pressão
count,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,944.000000,945.000000,945.000000
mean,13.820106,9.766138,2015.202116,11.561905,8.336089,81.403286,23.996437,85.502005,1016.540832
std,9.395129,4.415559,0.401791,6.926263,2.776005,68.414114,2.163369,8.555805,7.117770
min,1.000000,1.000000,2015.000000,0.000000,1.068333,1.448179,15.609589,46.355556,937.534333
25%,5.000000,12.000000,2015.000000,6.000000,6.494444,41.022403,22.963406,81.287356,1014.801833
50%,12.000000,12.000000,2015.000000,12.000000,8.402778,63.557747,24.093095,87.144444,1018.027500
75%,22.000000,12.000000,2015.000000,18.000000,10.297222,87.846917,25.327309,91.527778,1020.723333
max,31.000000,12.000000,2016.000000,23.000000,15.313889,356.553715,30.140625,100.231884,1023.726500


In [79]:
Mucuri[Mucuri[['dia', 'mês', 'ano', 'hora', 'v_anemo2', 'dir_1', 'temp', 'umid', 'pressão']].isnull()].head()

,Data,dia,mês,ano,hora,v_anemo2,dir_1,temp,umid,pressão
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
Mucuri.dropna(inplace = True)
Mucuri.isnull().sum()

Data        0
dia         0
mês         0
ano         0
hora        0
v_anemo2    0
dir_1       0
temp        0
umid        0
pressão     0
dtype: int64

In [85]:
x_array = Mucuri[['dia','mês','ano','hora','dir_1','temp','umid','pressão']].values
x_array.shape

(944, 8)

In [86]:
y_array = list(Mucuri['v_anemo2'].values)
y_array

[13.0121387471,
 12.7260869621,
 12.0811111291,
 11.6472222434,
 11.0644444413,
 10.3244444529,
 9.86277778943,
 9.59888886876,
 9.48888888359,
 7.50222221745,
 7.08111110263,
 8.00333341493,
 8.14944446617,
 8.96277779208,
 7.67277780374,
 6.33166668415,
 4.58717949727,
 4.96111110714,
 5.65944445001,
 5.41777779394,
 7.3039106044,
 8.60277779632,
 11.7061110814,
 12.3050000509,
 12.0872222159,
 11.3416666826,
 10.6938889398,
 9.51944443915,
 8.94000002013,
 8.10500001642,
 7.63777777884,
 6.66111112171,
 4.80111111138,
 3.71777777407,
 2.14166665673,
 2.77055555979,
 2.74944443835,
 3.91222221851,
 5.53222223123,
 6.35166666243,
 4.57777777645,
 3.91888887088,
 4.28111111323,
 3.59999999073,
 3.90555555423,
 7.15611111323,
 8.16722222169,
 8.54777779844,
 9.71222220527,
 10.480555582,
 10.2833333386,
 9.58777776824,
 9.03777779738,
 7.9066666868,
 7.55222228368,
 6.91333331532,
 6.85666667885,
 6.58833333916,
 6.3805555635,
 6.20388890372,
 6.58888887829,
 7.84388889207,
 8.235555542

In [87]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(9, activation='relu', input_shape=(8,)),
    tf.keras.layers.Dense(9, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear'),
  ])

  model.compile(optimizer='adam',
                loss='mse',
                metrics=['mae'])
  return model

In [94]:
model = get_compiled_model()
model.fit(x_array, y_array, epochs=5)

Epoch 1/5
944/944 [==============================] - 0s 188us/sample - loss: 248727.0254 - mean_absolute_error: 478.3101
Epoch 2/5
944/944 [==============================] - 0s 50us/sample - loss: 13460.5397 - mean_absolute_error: 92.9492
Epoch 3/5
944/944 [==============================] - 0s 51us/sample - loss: 73.4441 - mean_absolute_error: 7.6017
Epoch 4/5
944/944 [==============================] - 0s 53us/sample - loss: 34.9670 - mean_absolute_error: 5.0237
Epoch 5/5
944/944 [==============================] - 0s 51us/sample - loss: 12.1698 - mean_absolute_error: 2.5937


In [95]:
model.evaluate(x_array, y_array)

944/944 [==============================] - 0s 106us/sample - loss: 12.2166 - mean_absolute_error: 2.5663


[12.216572721125717, 2.5662804]

In [96]:
model.predict(x_array) 

array([[10.357175 ],
       [10.169236 ],
       [10.116422 ],
       [ 9.6682825],
       [ 9.558969 ],
       [ 9.39391  ],
       [ 9.251119 ],
       [ 9.175611 ],
       [ 9.261117 ],
       [ 9.292684 ],
       [ 5.148824 ],
       [ 5.567884 ],
       [ 5.5616126],
       [ 5.2555366],
       [ 5.343248 ],
       [ 5.445562 ],
       [ 5.6474433],
       [ 6.209329 ],
       [ 6.724542 ],
       [ 7.8921065],
       [ 9.048654 ],
       [ 9.159844 ],
       [ 8.981111 ],
       [ 9.039727 ],
       [ 9.229844 ],
       [ 9.256406 ],
       [ 9.349946 ],
       [ 9.334249 ],
       [ 9.416047 ],
       [ 9.494855 ],
       [ 9.631234 ],
       [ 9.483624 ],
       [ 9.164948 ],
       [ 9.356694 ],
       [ 5.659593 ],
       [ 5.2317443],
       [16.27714  ],
       [15.987237 ],
       [15.85032  ],
       [16.357565 ],
       [15.882813 ],
       [16.311575 ],
       [ 5.873937 ],
       [ 6.7090316],
       [ 9.289525 ],
       [ 9.415727 ],
       [ 9.596169 ],
       [ 9.58